In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from plyfile import PlyData
import zmsh

Download the Stanford bunny image.

In [ ]:
!wget --no-clobber http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz
!tar -zxvf bunny.tar.gz --wildcards --no-anchored '*bun_zipper_res2.ply'

Load in the model using the `plyfile` package and extract the points and triangles.

In [ ]:
filename = "bunny/reconstruction/bun_zipper_res2.ply"
with open(filename, "r") as bunny_file:
    model = PlyData.read(bunny_file)

vertices = model.elements[0]
x, y, z = vertices["x"], vertices["y"], vertices["z"]
points = np.column_stack((x, y, z))
triangles = np.array([tuple(elt) for elt in model.elements[1].data["vertex_indices"]])

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.view_init(elev=120)
ax.plot_trisurf(x, y, z, triangles=triangles);

Make a mapping from vertices to triangles.

In [ ]:
cotriangles = [[] for index in range(len(points))]
for index, triangle in enumerate(triangles):
    for vertex in triangle:
        cotriangles[vertex].append(index)

We'll pick out the first two vertices of triangle 0 to focus on.

In [ ]:
triangles[0]

In [ ]:
vertex0, vertex1 = triangles[0][:2]
vertex0, vertex1

In [ ]:
triangle_ids = list(set(cotriangles[vertex0]).union(cotriangles[vertex1]))
patch = triangles[triangle_ids]
patch

In [ ]:
vertex_ids = np.unique(patch.flatten())
vertex_ids

To simplify things, we'll renumber the vertices on this patch.

In [ ]:
id_map = np.vectorize({idx: val for val, idx in enumerate(vertex_ids)}.get)

In [ ]:
patch = id_map(patch)
vtx0 = id_map(vertex0)
vtx1 = id_map(vertex1)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
colors = ["tab:green" for index in range(len(vertex_ids))]
colors[vtx0] = "tab:orange"
colors[vtx1] = "tab:orange"
ax.plot_trisurf(x[vertex_ids], y[vertex_ids], z[vertex_ids], triangles=patch)
ax.scatter(x[vertex_ids], y[vertex_ids], z[vertex_ids], color=colors);

In [ ]:
copatch = [[] for index in range(len(vertex_ids))]
for index, triangle in enumerate(patch):
    for vertex in triangle:
        copatch[vertex].append(index)

copatch

In [ ]:
Q0 = zmsh.simplification.compute_qmatrix(points[vertex_ids[patch[copatch[vtx0]]]])
Q1 = zmsh.simplification.compute_qmatrix(points[vertex_ids[patch[copatch[vtx1]]]])
Q0, Q1

In [ ]:
Q = Q0 + Q1
Q[0, 0] = 1
Q[0, 1:] = 0
f = np.array([1, 0, 0, 0])
r = np.linalg.solve(Q, f)[1:]
r, points[triangles[0][0]]

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
colors = ["tab:green" for index in range(len(vertex_ids))]
colors[vtx0] = "tab:orange"
colors[vtx1] = "tab:orange"
ax.plot_trisurf(x[vertex_ids], y[vertex_ids], z[vertex_ids], triangles=patch)
ax.scatter(x[vertex_ids], y[vertex_ids], z[vertex_ids], color=colors)
ax.scatter([r[0]], [r[1]], [r[2]], color="black");

In [ ]:
d0, d1, d2 = zmsh.polytopal.from_simplicial(patch)

In [ ]:
P0 = np.eye(len(vertex_ids), dtype=np.int8)
P0[vtx0, [vtx0, vtx1]] = (+1, +1)
P0[vtx1, :] = 0
P0

In [ ]:
e1 = P0 @ d1
e1.T

In [ ]:
for index0, col0 in enumerate(e1.T):
    for index1, col1 in enumerate(e1.T):
        if (np.array_equal(col0, col1) or np.array_equal(col0, -col1)) and (index0 != index1) and (not np.all(col0 == 0)):
            print(index0, index1)

In [ ]:
zmsh.polytopal.merge([e1, d2], face_ids=[3, 13])